<center><h1>Enhao_Wang_HW4_Code & Explanation</h1></center>

In [3]:
from bs4 import BeautifulSoup
import re
import time
import pandas as pd
import numpy as np
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys

# 1. Dataset 1 - Houses around USC

Objective: download data of houses around USC from [apartments.com](https://www.apartments.com/off-campus-housing/ca/los-angeles/university-of-southern-california-university-park-campus/). House data includes house name, location, price and number of beds. Number of beds maybe just an extension of data downloading work, not used in the research. 

Basic idea: house data is from many pages from apartments.com(each page displays 25 houses). First get all the JavaScript content of the webpages, Then use beautifulsoups to retrive the information. request package can't obtain JS webpages, so I used selenium instead. Instructions on how to use selenium is included in README.txt.

The later datasets are all based on this dataset, that is, these datasets 2,3,4,5 are to find other information of houses in this dataset.

In [6]:
# given a list of webpages, obtain the information of all houses in these pages.
def find_house_info(webpages):
    names,locations,prices,beds=[],[],[],[] # four columns: name, location, price, number of beds
    for i in range(len(webpages)):
        soup = BeautifulSoup(webpages[i], 'html.parser')

        # locate all the houses in a page
        houses = soup.find_all('li', class_='mortar-wrapper') 

        for house in houses:
            tags=house.find_all('span', class_='js-placardTitle title')
            name=tags[0].text

            tags=house.find_all('div', class_='property-address js-url') or house.find_all('p', class_='property-address js-url')
            if len(tags)==1:
                location=tags[0]['title']
            else:
                location=tags[0].text+', '+tags[1].text

            tags=house.find_all('p', class_='property-pricing') or house.find_all('span', class_='property-rents') or house.find_all('div', class_='price-range')
            price=tags[0].text

            tags=house.find_all('p', class_='property-beds') or house.find_all('span', class_='property-beds') or house.find_all('div', class_='bed-range')
            bed=tags[0].text
            
            names.append(name)
            locations.append(location)
            
            # for prices in a range, take its middle value
            price_num=re.findall(r'(?:\d+,)?\d+',price) # find numbers in the price
            if len(price_num)==0: # Call for rent, no price provided
                prices.append(price)
            elif len(price_num)>2: # More than 2 numbers in its price, which is not normal
                print('Attention on price',name)
            else: # 1 or 2 numbers in the price
                for i in range(len(price_num)):
                    price_num[i]=int(''.join(price_num[i].split(','))) # e.g. convert str '2,500' to int 2500
                prices.append(int(sum(price_num)/len(price_num))) # take average
            
            beds.append(bed)

    house_info={'Name':names,'Location':locations,'Price(in $)':prices,'Number of beds':beds}
    house_info=pd.DataFrame(house_info)
    return house_info

def get_dataset_1(sample_5:bool):
    driver = webdriver.Chrome()  
    driver.maximize_window() 
    # first get the first page
    url = "https://www.apartments.com/off-campus-housing/ca/los-angeles/university-of-southern-california-university-park-campus/"

    driver.get(url) # open website

    time.sleep(2)  # wait for 2 seconds for the page to load

    page1=driver.page_source # get html content

    webpages=[page1]

    # if only 5 samples, only the first page is enough, so don't need to get other pages and just return the first 5 rows of result data frame from the first page.
    if sample_5:
        return find_house_info(webpages).iloc[:5,:]
    
    soup = BeautifulSoup(page1, 'html.parser')
    page_range = soup.find('span', {'class': 'pageRange'}) # used to find the number of pages in total

    if page_range: # if page_range doesn't exist, this means there is only one page.
        page_num=int(re.findall(r'Page 1 of (\d+)',page_range.text)[0]) # get the number of pages in total
        
        for i in range(2,page_num+1):
            url = "https://www.apartments.com/off-campus-housing/ca/los-angeles/university-of-southern-california-university-park-campus/"+str(i)
            driver.get(url)
            # read the page content again and again until the program assures it is the right page. 
            while 1:
                page = driver.page_source
                soup=BeautifulSoup(page,'html.parser')
                current_page=soup.find('span', {'class': 'pageRange'})
                current_page_num=int(re.findall(r'Page (\d+) of ',current_page.text)[0])
                if current_page_num==i: # i=right page number
                    break
            webpages.append(page)
            
    driver.quit()

    house_info=find_house_info(webpages)
    house_info.to_csv('house_info.csv', index=False) # save the data frame as a .csv file.

In [7]:
# get all the data and save them.
get_dataset_1(False)

In [86]:
# run the code here to get a preview of first 5 samples in dataset 1.
print('First 5 entries of dataset 1:')
get_dataset_1(True)

First 5 entries of dataset 1:


,Name,Location,Price(in $),Number of beds
0,Jasper,"2528 S Grand Ave, Los Angeles, CA 90007",3150,Studio - 2 Beds
1,Private Suite - Co-living Life,"1430 W 37th St, Los Angeles, CA 90018",1269,1-6 Beds
2,THEA at Metropolis,"1000 W 8th St, Los Angeles, CA 90017",13733,Studio - 3 Beds
3,Josephine DTLA,"801 S Hope St, Los Angeles, CA 90017",4180,1-3 Beds
4,Beaudry,"960 W 7th St, Los Angeles, CA 90017",6302,Studio - 3 Beds


# 2. Dataset 2 - Coordinates of Houses

Actually as mentioned in Homework 3, the 'true' dataset 2 is the next one, because it needs the information of this dataset. But the data sources are different, so I just let this also be a dataset.

Objective: get the latitude and longitude coordinates for each house in dataset 1, from [gps-coordinates.net](https://www.gps-coordinates.net/). 

Idea: use selenium to simulate what we do on this website. In the 'Address' block, input the location of a house, then click 'Get GPS Coordinates', lastly the coordinates is shown in 'Lat,Long' block, just fetch the content inside it. Repeat for each house. 

In case of having some unexpected errors, I splitted the whole dataset(700 entries) into 7 small ones(100 entries each), and find information for them one by one. After one small dataset is completed, save as a .csv file to make sure the data downloaded is safe. The program will print out which dataset occurs error, and then I can start from this dataset to repeat the work. After all these small datasets finish, concatinate them.

The location of each house from dataset 1 is in a formal and complete format, so there will be no problem that the input location is misleading and the website finds a wrong location for it. This is also true for later datasets.

In [35]:
# here are three nested functions. Their functions:
# find_coordinates: given a location, find and return it's lat and long
# get_coordinates: given a dataset(with a couple of houses), find and return lat and long information of the houses in it
# get_dataset_2: given a list of datasets, find and return lat and long information of the houses in them, and save as .csv files

def find_coordinates(driver,location):
    input_blk=driver.find_element(By.XPATH,'//*[@id="address"]') # 'Address' input block
    # clear the content in the input block. 
    # The website will stupidly input some places automatically after once clearing, so repeat clearing 10 times to make sure the input block is empty before the correct location is inputed.
    for i in range(10):
        input_blk.send_keys(Keys.CONTROL, "a")
        input_blk.send_keys(Keys.DELETE)
        time.sleep(0.1)
    
    input_blk.send_keys(location) # input the location
    time.sleep(1)

    get_crd_btn=driver.find_element(By.XPATH,'//*[@id="wrap"]/div[2]/div[3]/div[1]/form[1]/div[2]/div/button') # 'Get GPS Coordinates' button
    get_crd_btn.click() # click on it
    time.sleep(1)

    coordinate_btn=driver.find_element(By.XPATH,'//*[@id="latlong"]') # 'Lat,Long' block, containing the coordinates for this location
    return coordinate_btn.get_attribute('value') # return the value inside 'Lat,Long' block

def get_coordinates(driver,dataset):
    locations=dataset['Location'].tolist()
    names=dataset['Name'].tolist()
    coordinate_df=[]

    for name,location in zip(names,locations):
        coordinate=find_coordinates(driver,location)
        # convert the information in 'Lat,Long' block to latitude and longitude numbers
        lat,long=coordinate.split(',')
        lat,long=float(lat),float(long)
        coordinate_df.append([name,location,lat,long]) # append the information of each row

    coordinate_df=pd.DataFrame(coordinate_df)
    coordinate_df.columns=['Name','Location','Latitude','Longitude']

    return coordinate_df

def get_dataset_2(small_datasets,start,sample_5):
    driver = webdriver.Chrome()  
    driver.implicitly_wait(10)
    driver.maximize_window() 

    url = "https://www.gps-coordinates.net/"

    driver.get(url)
    time.sleep(2)

    # I runned the code without accepting the cookie of this website. If it causes some trouble, enable the following code to accept cookie.
    # cookie_btn=driver.find_element(By.XPATH,'/html/body/div[1]/div/a[1]')
    # cookie_btn.click()
    # time.sleep(1)

    if sample_5:
        return get_coordinates(driver,small_datasets[0].iloc[:5,:]) # just give the first 5 entries and return the data frame built for these 5 entries.
    
    for i in range(start,len(small_datasets)):
        print(f'The {i+1} small dataset begins!') # indicate the index of small dataset running.

        coordinate_df=get_coordinates(driver,small_datasets[i])
        coordinate_df.to_csv(f'coordinates{i+1}.csv',index=False) # for each small dataset, save as .csv file 

In [5]:
house_info=pd.read_csv('house_info.csv')
house_info=house_info.drop(['Price(in $)','Number of beds'],axis=1) # delete columns that are not needed
small_datasets=np.array_split(house_info, 7) # split the whole dataset into 7. Can be set to any number.

In [ ]:
# get all the data and save them.
get_dataset_2(small_datasets,0,False) # If an error occurs on the ith dataset, set the second parameter 'start' to i-1 and run this again.

In [87]:
# get a preview of first 5 samples in dataset 2.
print('First 5 entries of dataset 2:')
get_dataset_2(small_datasets,0,True)

First 5 entries of dataset 2:


,Name,Location,Latitude,Longitude
0,Jasper,"2528 S Grand Ave, Los Angeles, CA 90007",34.027057,-118.271787
1,Private Suite - Co-living Life,"1430 W 37th St, Los Angeles, CA 90018",34.020882,-118.301084
2,THEA at Metropolis,"1000 W 8th St, Los Angeles, CA 90017",34.049487,-118.265004
3,Josephine DTLA,"801 S Hope St, Los Angeles, CA 90017",34.046531,-118.259441
4,Beaudry,"960 W 7th St, Los Angeles, CA 90017",34.049566,-118.260700


In [88]:
coordinate_info=pd.concat([pd.read_csv(f'coordinates{i+1}.csv') for i in range(len(small_datasets))]) # concatinate all the small dataset into a big one.
coordinate_info.to_csv('coordinates.csv',index=False) # dataset 2 completed!

# 3. Dataset 3 - Distances of Houses from USC (Not used)

Objective: using coordinates in dataset 2, get the distances(in meter) from USC(coordinates: -118.2851, 34.0224, from Google) for each house in dataset 1, from [openrouteservice](https://openrouteservice.org/). 

This website provides API for a lot of functions. Given the latitude and longitude of two places, it can calculate the route, distance and time spent on the route with a paticular transportation(like driving, biking, on foot) between them.

Actually it's not accurate to say this API calculates the distance from a house to USC, because indeed it calculates the length of the route from one house to USC, not the length of a point-to-point straight line. Although there is some positive relationship between the length of route and the distance between two points, I have to admit that this will yield inaccurate distance information for later research. 

Idea: the code of finding distance is already provided by the website. I can just copy it and do some modification so it can iterate for all houses.

Also split the whole dataset into 7 small ones, as dataset 2 does.

Update on April 21: This dataset is not used, see the next dataset 3. 

In [62]:
# similar to dataset 2, get_distances returns a distance data frame for all houses in the input dataset, 
# get_dataset_3 get and save the distance datasets given a list of small datasets.
def get_distances(dataset):
    headers = {
        'Accept': 'application/json, application/geo+json, application/gpx+xml, img/png; charset=utf-8',
        'Authorization': '5b3ce3597851110001cf62484fbf933ae82d4339b4a2702dc788e033',
        'Content-Type': 'application/json; charset=utf-8'
        }
    
    distances=[]
    for index,row in dataset.iterrows():
        time.sleep(2) # wait a period between each queries, or it will be blocked if the frequency is too high.
        lat=float(row['Latitude'])
        long=float(row['Longitude'])
        body = {"locations":[[long,lat],[-118.2851,34.0224]],"destinations":[1],"metrics":["distance"]}   
        call = requests.post('https://api.openrouteservice.org/v2/matrix/driving-car', json=body, headers=headers)
        distances.append(call.json()['distances'][0][0])

    dataset = dataset.copy()
    dataset.loc[:,'Distance(meter)']=distances
    dataset=dataset.drop(['Latitude','Longitude'],axis=1) # delete columns that are not needed
    return dataset    
        
def get_dataset_3(small_datasets,start,sample_5):
    if sample_5:
        return get_distances(small_datasets[0].iloc[:5,:]) # just give the first 5 entries and return the data frame built for these 5 entries.
    for i in range(start,len(small_datasets)):
        print(f'The {i+1} small dataset begins!') # indicate the index of small dataset running.
        distance_df=get_distances(small_datasets[i])
        distance_df.to_csv(f'distances{i+1}.csv',index=False) # for each small dataset, save as .csv file 

In [41]:
coordinate_info=pd.read_csv('coordinates.csv')
small_datasets=np.array_split(coordinate_info, 7) # split the whole dataset into 7.

In [ ]:
# get all the data and save them.
get_dataset_3(small_datasets,0,False) # If an error occurs on the ith dataset, set the second parameter 'start' to i-1 and run this again.

In [63]:
# get a preview of first 5 samples in dataset 3.
print('First 5 entries of dataset 3:')
get_dataset_3(small_datasets,0,True)

First 5 entries of dataset 3:


,Name,Location,Distance(meter)
0,Jasper,"2528 S Grand Ave, Los Angeles, CA 90007",2187.21
1,Private Suite - Co-living Life,"1430 W 37th St, Los Angeles, CA 90018",2054.31
2,THEA at Metropolis,"1000 W 8th St, Los Angeles, CA 90017",4236.31
3,Josephine DTLA,"801 S Hope St, Los Angeles, CA 90017",4519.28
4,Beaudry,"960 W 7th St, Los Angeles, CA 90017",4825.47


In [42]:
distance_info=pd.concat([pd.read_csv(f'distances{i+1}.csv') for i in range(len(small_datasets))]) # concatinate all the small dataset into a big one.
distance_info.to_csv('distances.csv',index=False) # dataset 3 completed!

# 3. Dataset 3 - Distances of Houses from USC (Using)

I just found another API for dataset 3 one day before due day of Homework 4. This API is from [rapidapi.com](https://rapidapi.com/tierisvalente-XBIB_hr-jt2/api/distance-calculator8/), with 1000 queries for free one month. I just tried to query for my 700 houses once, and luckily it successed. This API is more appropriate for calculating the point-to-point distance between two locations. So I decided to use it instead of the previous one. This is a change in my final project plan. The dataset 3 submitted is from this API, not the previous one. I also include the previous one above because I think it's also part of my work, and it has good extensibiltiy, although it's not used now.

Idea: the code of finding distance is already provided by the website. I can just copy it and do some modification so it can iterate for all houses.

Also split the whole dataset into 7 small ones, as dataset 2 does.

In [44]:
# similar to dataset 2, get_distances returns a distance data frame for all houses in the input dataset, 
# get_dataset_3 get and save the distance datasets given a list of small datasets.
def get_distances(dataset):
    headers = {
	"X-RapidAPI-Key": "8c6f548055msh1ff1bc25a4c2bfep19caa5jsn43c9ea274e19",
	"X-RapidAPI-Host": "distance-calculator8.p.rapidapi.com"
    }
    url = "https://distance-calculator8.p.rapidapi.com/calc"
    
    distances=[]
    for index,row in dataset.iterrows():
        time.sleep(2) # wait a period between each queries, or it will be blocked if the frequency is too high.
        lat=row['Latitude']
        long=row['Longitude']
        querystring = {"startLatitude":str(lat),"startLongitude":str(long),"endLatitude":"34.0224","endLongitude":"-118.2851"} 
        response = requests.request("GET", url, headers=headers, params=querystring)
        distances.append(response.json()['body']['distance']['meters'])

    dataset = dataset.copy()
    dataset.loc[:,'Distance(meter)']=distances
    dataset=dataset.drop(['Latitude','Longitude'],axis=1) # delete columns that are not needed
    return dataset    
        
def get_dataset_3(small_datasets,start,sample_5):
    if sample_5:
        return get_distances(small_datasets[0].iloc[:5,:]) # just give the first 5 entries and return the data frame built for these 5 entries.
    for i in range(start,len(small_datasets)):
        print(f'The {i+1} small dataset begins!') # indicate the index of small dataset running.
        distance_df=get_distances(small_datasets[i])
        distance_df.to_csv(f'distances{i+1}.csv',index=False) # for each small dataset, save as .csv file 

In [ ]:
house_info=pd.read_csv('coordinates.csv')
small_datasets=np.array_split(house_info, 7) # split the whole dataset into 7.

In [ ]:
# get all the data and save them.
get_dataset_3(small_datasets,0,False) # If an error occurs on the ith dataset, set the second parameter 'start' to i-1 and run this again.

In [61]:
# get a preview of first 5 samples in dataset 3.
print('First 5 entries of dataset 3:')
get_dataset_3(small_datasets,0,True)

First 5 entries of dataset 3:


,Name,Location,Distance(meter)
0,Jasper,"2528 S Grand Ave, Los Angeles, CA 90007",1331.707427
1,Private Suite - Co-living Life,"1430 W 37th St, Los Angeles, CA 90018",1482.784033
2,THEA at Metropolis,"1000 W 8th St, Los Angeles, CA 90017",3535.644599
3,Josephine DTLA,"801 S Hope St, Los Angeles, CA 90017",3576.366130
4,Beaudry,"960 W 7th St, Los Angeles, CA 90017",3765.601604


In [ ]:
distance_info=pd.concat([pd.read_csv(f'distances{i+1}.csv') for i in range(len(small_datasets))]) # concatinate all the small dataset into a big one.
distance_info.to_csv('distances.csv',index=False) # dataset 3 completed!

# 4. Dataset 4 & 5 - Crimes of Houses

Objective: get the crime records within a distance of 1/2 miles(a circular range) and a time period(28 days) for each house in dataset 1, from [crimemapping.com](https://www.crimemapping.com/map/ca/losangeles#). 

Dataset 4 is all the crime records for houses, including crime type, description, location and date. This dataset is not included in Homework 3 because I just decided to fetch these data when I was writing the code, after Homework 3 handed in. Probably this dataset will not be used in final research, but it might have other usages later.

Dataset 5 is a satistics for crime records for all houses regarding to crime type. Each record of a crime has a type. In this dataset, each entry(house) will include the total number of crimes around it, and the number of each type of crime. The sum of all types is equal to the total number of crimes. 

Idea: use selenium to simulate what we do on this website. Click on 'Where' and 'When' buttons in the left will pop up filter panels. Input the location in 'where' panel, and select the time from how far back we want to search for crime records in 'when' panel. Then click the 'Apply' button in the 'where' panel to load the crimes, and click 'Report' button in the left to get the records. For each page in the records, click right arrow button to get to the next page, lastly click 'Back to Map' button to go back when all the records are fetched. Repeat for each house. 

Also split the whole dataset into 7 small ones, as dataset 2 does.

In [47]:
# here are three nested functions. Their functions:
# find_crimes: given a location, find and return the records of crimes around it
# get_crime_info: given a dataset(with a couple of houses), find and return the records of crimes around it
# get_dataset_4: given a list of datasets, find and return the records of crimes around it, and save as .csv files

def find_crimes(driver,loc):
    where_btn=driver.find_element(By.XPATH,'//*[@id="filtersWhere"]') # 'Where' button
    driver.execute_script('arguments[0].click()',where_btn) # click on it
    time.sleep(1)

    input_blk=driver.find_element(By.XPATH,'//*[@id="locationSearchWherePanel"]')
    input_blk.clear() # clear the content in the input block. 
    time.sleep(1)
    input_blk.send_keys(loc) # input the location
    time.sleep(1)

    apply_btn=driver.find_element(By.XPATH,'//*[@id="wherePanel"]/table/tbody/tr/td[1]/a') # 'Apply' button
    driver.execute_script('arguments[0].click()',apply_btn) # click on it
    time.sleep(3)

    report_btn=driver.find_element(By.XPATH,'//*[@id="displayReports"]') # 'Report' button
    # if no crimes, 'Report' button is not clickable. So first check if the 'Report' button is clickable.
    if report_btn.get_attribute('class')=='viewReportToolbar viewReportToolbarDisabled':
        driver.execute_script('arguments[0].click()',where_btn) # click on 'Where' button again to close the 'where' panel
        time.sleep(1)
        return []
    driver.execute_script('arguments[0].click()',report_btn) # click on 'Report'
    time.sleep(5)

    crimes=[]
    type_mapping=['Arson',
	 	'Assault',
	 	'Burglary',
	 	'Disturbing the Peace',
	 	'Drugs/Alcohol Violations',
	 	'DUI',
	 	'Fraud',
	 	'Homicide',
	 	'Motor Vehicle Theft',
	 	'Robbery',
	 	'Sex Crimes',
	 	'Theft/Larceny',
	 	'Vandalism',
	 	'Vehicle Break-In/Theft',
	 	'Weapons'] # list index mapping with crime type
    
    incident_num='0'
    # iterate for all pages
    while 1:
        # read the page content again and again until the program assures it is the right page.
        # incident_num is an unique number for each crime. 
        # crime[3].text is the first incident_num for each page, so if it's not equal to the crime[3].text of the last page, 
        # it means this is a new page so we are at the right page.
        while 1:
            html=driver.page_source
            soup = BeautifulSoup(html, 'html.parser')
            crime=soup.find_all('td',{'role':'gridcell'})
            if len(crime)>0 and crime[3].text!=incident_num:
                break
        incident_num=crime[3].text

        num_of_crimes=int(len(crime)/7) # one crime record has 7 column(td)s
        # crime type, description, location and date
        for i in range(num_of_crimes):
            crimes.append((type_mapping[int(re.findall(r'(\d+).svg',crime[1+7*i].find_all('img')[0]['src'])[0])-1],crime[2+7*i].text,crime[4+7*i].text,crime[6+7*i].text))

        next_btn=driver.find_element(By.XPATH,'//*[@id="CrimeIncidents"]/div[3]/a[3]') # right arrow button to jump to the next page
        if next_btn.get_attribute('class')=='k-link k-state-disabled': # right arrow button is not clickable, which means it is the last page.
            break
        driver.execute_script('arguments[0].click()',next_btn) # click on it
        # time.sleep(4)

    back_btn=driver.find_element(By.XPATH,'//*[@id="reportContainer"]/a[3]') # 'Back to Map' button
    driver.execute_script('arguments[0].click()',back_btn) # click on it to return to the map
    time.sleep(2)
    
    return crimes

def get_crime_info(driver,dataset):
    locations=dataset['Location'].tolist()
    names=dataset['Name'].tolist()

    crime_info=[]
    for location in locations:
        crime_info.append(find_crimes(driver,location))

    # dataset 4
    crime_info_df=[]
    for name,location,crimes in zip(names,locations,crime_info):
        if crimes==[]: # no crime record
            crime_info_df.append([name,location,'No Crime','No Crime','No Crime','No Crime']) 
        for crime in crimes:
            crime_info_df.append([name,location,crime[0],crime[1],crime[2],crime[3]]) # append the information of each row

    crime_info_df=pd.DataFrame(crime_info_df)
    crime_info_df.columns=['House Name','House Location','Crime Type','Crime Description','Crime Location','Crime Date']
    
    # dataset 5
    crime_count_df=[]
    for name,location,crimes in zip(names,locations,crime_info):
        crime_dict={
            'Name':name,'Location':location,'Total':len(crimes),
            'Arson':0,	
            'Assault':0,	
            'Burglary':0,	
            'Disturbing the Peace':0,	
            'Drugs/Alcohol Violations':0,	
            'DUI':0,	
            'Fraud':0,	
            'Homicide':0,	
            'Motor Vehicle Theft':0,	
            'Robbery':0,	
            'Sex Crimes':0,	
            'Theft/Larceny':0,	
            'Vandalism':0,	
            'Vehicle Break-In/Theft':0,	
            'Weapons':0}
        for crime in crimes:
            crime_dict[crime[0]]+=1
        crime_count_df.append(crime_dict) # append the information of each row

    crime_count_df=pd.DataFrame(crime_count_df)
    
    return crime_info_df,crime_count_df

def get_dataset_4(small_datasets,start,sample_5:bool):
    driver = webdriver.Chrome()  
    driver.implicitly_wait(10)
    driver.maximize_window() 

    url = "https://www.crimemapping.com/map/ca/losangeles#"

    driver.get(url)
    time.sleep(5)

    # if the screen is small, the 'Report' button might not appear on the screen, so we need to enable the following code to shrink the webpage.  
    # driver.execute_script("document.body.style.zoom='50%'")
    # time.sleep(1)

    when_btn=driver.find_element(By.XPATH,'//*[@id="filtersWhen"]') # 'When' button
    driver.execute_script('arguments[0].click()',when_btn) # click on it
    time.sleep(1)

    four_week_btn=driver.find_element(By.XPATH,'//*[@id="whenPanel"]/ul/li[7]/div/a') # 'Previous 4 Weeks' button
    driver.execute_script('arguments[0].click()',four_week_btn) # click on it to choose the record 4 weeks from now
    time.sleep(1)

    if sample_5:
        return get_crime_info(driver,small_datasets[0].iloc[:5,:]) # just give the first 5 entries and return the data frame built for these 5 entries.

    for i in range(start,len(small_datasets)):
        print(f'The {i+1} small dataset begins!') # indicate the index of small dataset running.

        crime_info_df,crime_count_df=get_crime_info(driver,small_datasets[i]) 
        
        # for each small dataset, save as .csv file
        crime_info_df.to_csv(f'crime_info{i+1}.csv',index=False)
        crime_count_df.to_csv(f'crime_count{i+1}.csv',index=False)

In [46]:
house_info=pd.read_csv('house_info.csv')
house_info=house_info.drop(['Price(in $)','Number of beds'],axis=1) # delete columns that are not needed
small_datasets=np.array_split(house_info, 7) # split the whole dataset into 7

In [ ]:
# get all the data and save them.
get_dataset_4(small_datasets,0,False) # If an error occurs on the ith dataset, set the second parameter 'start' to i-1 and run this again.

Actually for dataset 4, input the first 5 samples will return all the crime records for these 5 houses. So the result of first 5 samples will have many rows.

In [47]:
# get a preview of first 5 samples in dataset 4.
crime_info_df,crime_count_df=get_dataset_4(small_datasets,0,True)
print('First 5 entries of dataset 4:')
crime_info_df

First 5 entries of dataset 4:


,House Name,House Location,Crime Type,Crime Description,Crime Location,Crime Date
0,Jasper,"2528 S Grand Ave, Los Angeles, CA 90007",Assault,SPOUSAL (cohab) ABUSE - AGGRAVATED ASSAULT,29TH ST & FIGUEROA,4-18-2023 5:00 PM
1,Jasper,"2528 S Grand Ave, Los Angeles, CA 90007",Assault,ASSAULT WITH DEADLY WEAPON (ADW) (likely to pr...,MAPLE AV & ADAMS BL,4-17-2023 11:50 PM
2,Jasper,"2528 S Grand Ave, Los Angeles, CA 90007",Theft/Larceny,THEFT FROM PERSON,GRAND AV & 33RD ST,4-17-2023 4:45 PM
3,Jasper,"2528 S Grand Ave, Los Angeles, CA 90007",Motor Vehicle Theft,"VEHICLE, STOLEN",3200 BLOCK S MAIN ST,4-17-2023 1:00 AM
4,Jasper,"2528 S Grand Ave, Los Angeles, CA 90007",Vehicle Break-In/Theft,BURGLARY FROM VEHICLE,2800 BLOCK MAPLE AV,4-15-2023 2:38 AM
...,...,...,...,...,...,...
735,Beaudry,"960 W 7th St, Los Angeles, CA 90017",Theft/Larceny,THEFT-GRAND (plain theft-grand) $950.01 & over...,1200 BLOCK W 7TH ST,3-24-2023 12:00 PM
736,Beaudry,"960 W 7th St, Los Angeles, CA 90017",Theft/Larceny,SHOPLIFTING - PETTY THEFT ($950 & under),700 BLOCK W 7TH ST,3-24-2023 11:15 AM
737,Beaudry,"960 W 7th St, Los Angeles, CA 90017",Motor Vehicle Theft,"VEHICLE, STOLEN",1000 BLOCK W 7TH ST,3-24-2023 9:43 AM
738,Beaudry,"960 W 7th St, Los Angeles, CA 90017",Vehicle Break-In/Theft,BURGLARY FROM VEHICLE,1000 BLOCK WILSHIRE BL,3-24-2023 9:00 AM


In [48]:
# get a preview of first 5 samples in dataset 5.
print('First 5 entries of dataset 5:')
crime_count_df

First 5 entries of dataset 5:


,Name,Location,Total,Arson,Assault,Burglary,Disturbing the Peace,Drugs/Alcohol Violations,DUI,Fraud,Homicide,Motor Vehicle Theft,Robbery,Sex Crimes,Theft/Larceny,Vandalism,Vehicle Break-In/Theft,Weapons
0,Jasper,"2528 S Grand Ave, Los Angeles, CA 90007",53,0,9,3,0,0,0,0,0,14,4,0,16,0,7,0
1,Private Suite - Co-living Life,"1430 W 37th St, Los Angeles, CA 90018",46,0,7,10,0,0,0,0,0,13,5,1,3,0,7,0
2,THEA at Metropolis,"1000 W 8th St, Los Angeles, CA 90017",194,1,33,10,0,0,0,0,1,30,15,1,67,0,36,0
3,Josephine DTLA,"801 S Hope St, Los Angeles, CA 90017",245,1,34,7,0,0,0,0,0,41,21,0,92,0,49,0
4,Beaudry,"960 W 7th St, Los Angeles, CA 90017",202,0,34,8,0,0,0,0,1,32,18,1,67,0,41,0


In [23]:
crime_info=pd.concat([pd.read_csv(f'crime_info{i+1}.csv') for i in range(len(small_datasets))]) # concatinate all the small dataset into a big one.
crime_info.to_csv('crime_info.csv',index=False) # dataset 4 completed!

crime_count=pd.concat([pd.read_csv(f'crime_count{i+1}.csv') for i in range(len(small_datasets))]) # concatinate all the small dataset into a big one.
crime_count.to_csv('crime_count.csv',index=False) # dataset 5 completed!